In [54]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [55]:
#creating list to store the data
authors =[]
dates = []
statements = []
sources =[]
targets =[]


In [56]:
# creating a function to scrape a website
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num + '&pubdate=3months'
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')
    #getting the location of the information
    statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer'})#location of the author
    statement_quote = soup.find_all('div', attrs ={'class':'m-statement__quote'})#location of the quote
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#location of source
    target = soup.find_all('div',attrs={'class':'m-statement__meter'})#location of meter
    
    #loop for statement_footer
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name + ' '+last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)
    #loop for statement_quote
    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text)
    #loop for statement_meta
    for i in statement_meta:
        link3 =  i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)
    #loop for target
    for i in target:
        link4 = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)







In [75]:
#call the function 
n=5
for i in range(1,n):
    scrape_website(i)

In [77]:
# DATAFRAME
data = pd.DataFrame(columns = ['author' , 'statement' , 'source' , 'date' , 'target' ])
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets

#show the data
data

,author,statement,source,date,target
0,Gabrielle Lazor,There is “zero impact” from Alligator Alcatraz...,Ron DeSantis,"July 11, 2025",false
1,Louis Jacobson,A Congressional Budget Office estimate of peop...,Kevin Hassett,"July 11, 2025",barely-true
2,Amy Sherman,“ICE will now become the country’s largest fed...,Jon Favreau,"July 11, 2025",mostly-true
3,Will Doran,Former Gov. Roy Cooper is scheduled to speak a...,X posts,"July 10, 2025",false
4,Louis Jacobson,China doesn’t “have a lot of wind farms. … Ver...,Donald Trump,"July 10, 2025",pants-fire
...,...,...,...,...,...
199,Louis Jacobson,Kilmar Armando Abrego Garcia “had ‘MS-13’ on h...,Donald Trump,"April 30, 2025",pants-fire
200,Louis Jacobson,“We had a couple of states where gasoline was ...,Donald Trump,"April 24, 2025",false
201,Loreben Tuquero,“Gun violence is the leading killer of college...,Moms Demand Action for Gun Sense In America,"April 22, 2025",true
202,Claire Cranford,“The Simpsons” predicted Pope Francis’ 2025 de...,TikTok posts,"April 22, 2025",pants-fire


In [79]:
#creating a function to get a binary number from the target
def getBinaryNumTarget(text):
    if text == 'true':
        return 1
    else:
        return 0

In [81]:
#creating a function to get a binary  from the target
def getBinaryTarget(text):
    if text == 'true':
        return 'REAL'
    else:
        return 'FAKE'

In [83]:
#creating new columns in te dataframe for the 2 functions
data['BinaryTarget']= data['target'].apply(getBinaryTarget)
data['BinaryNumTarget']= data['target'].apply(getBinaryNumTarget)



In [85]:
data

,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Gabrielle Lazor,There is “zero impact” from Alligator Alcatraz...,Ron DeSantis,"July 11, 2025",false,FAKE,0
1,Louis Jacobson,A Congressional Budget Office estimate of peop...,Kevin Hassett,"July 11, 2025",barely-true,FAKE,0
2,Amy Sherman,“ICE will now become the country’s largest fed...,Jon Favreau,"July 11, 2025",mostly-true,FAKE,0
3,Will Doran,Former Gov. Roy Cooper is scheduled to speak a...,X posts,"July 10, 2025",false,FAKE,0
4,Louis Jacobson,China doesn’t “have a lot of wind farms. … Ver...,Donald Trump,"July 10, 2025",pants-fire,FAKE,0
...,...,...,...,...,...,...,...
199,Louis Jacobson,Kilmar Armando Abrego Garcia “had ‘MS-13’ on h...,Donald Trump,"April 30, 2025",pants-fire,FAKE,0
200,Louis Jacobson,“We had a couple of states where gasoline was ...,Donald Trump,"April 24, 2025",false,FAKE,0
201,Loreben Tuquero,“Gun violence is the leading killer of college...,Moms Demand Action for Gun Sense In America,"April 22, 2025",true,REAL,1
202,Claire Cranford,“The Simpsons” predicted Pope Francis’ 2025 de...,TikTok posts,"April 22, 2025",pants-fire,FAKE,0


In [87]:
data.to_csv('politifact_scraped_data.csv', index=False)